In [1]:
%pip install influxdb_client

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [7]:
import os
import pytz
import pandas as pd
from datetime import datetime, timedelta
from influxdb_client import InfluxDBClient

# 디바이스 ID와 위치를 매핑
location_mapping = {
    '24e124126d152919': 'indoor',
    '24e124126d152969': 'bottom_right_corner',
    '24e124128c067999': 'indoor',
    '24e124785c389818': 'bottom_left_corner',
    '24e124785c421885': 'top_right_corner'
}

# InfluxDB 설정 정보
url = "http://133.186.144.22:8086"
token = "BPJ1pnKvoaov4Tte971t0zpRSTUXNZvrshU7u3UPheAIsBeUJEFfbKjfsZjtwZmugkHJEGRW17lH4bR9ybanNQ=="
org = "smoothing"

# InfluxDB 클라이언트 생성
def create_client(url, token, org):
    return InfluxDBClient(url=url, token=token, org=org)

# 쿼리 실행 및 DataFrame으로 변환
def query_to_dataframe(client, query):
    result = client.query_api().query(query=query)
    results = []
    
    for table in result:
        for record in table.records:
            results.append({
                "time": record.get_time(),
                "value": record.get_value(),
                "place": record.values.get("place"),
                "location": record.values.get("location"),
                "device": record.values.get("device")
            })
    
    df = pd.DataFrame(results)
    df['time'] = df['time'].astype(str).str.replace(r'\+00:00$', '', regex=True)
    df = df.round(2) # 반올림 추가
    return df

# 데이터를 날짜를 지정하여 CSV 파일로 저장
def save_csv(df, file_pattern, directory):
    # 반올림 추가 
    df = df.round(2)
    # 경로가 존재하는지 확인하고, 없다면 생성
    if not os.path.exists(directory):
        os.makedirs(directory)
        
    current_date = datetime.now()
    previous_date = current_date - timedelta(days=1)
    filename = f"{directory}{previous_date.strftime(file_pattern)}"
    df.to_csv(filename, index=False)
    
# 온도 Data에서 'device' 열에 따라 'location' 열을 업데이트    
def update_location(df, location_mapping):
    df['location'] = df['device'].map(location_mapping)
    return df

### 쿼리 사용하여 조회

In [4]:
# 클라이언트 생성 및 쿼리 실행
client = create_client(url, token, org)

# 한국 시간대 설정
korea_tz = pytz.timezone('Asia/Seoul')

# 사용자로부터 입력받은 날짜와 시간
input_date_str = "2024-04-15 00:00:00"
input_format = "%Y-%m-%d %H:%M:%S"
sensor_name = "temperature"

# 입력 날짜를 datetime 객체로 변환
input_datetime = datetime.strptime(input_date_str, input_format)

# 한국 시간대로 localize
localized_kst = korea_tz.localize(input_datetime)

# UTC로 변환
start_time_utc = localized_kst.astimezone(pytz.utc)
end_time_utc = start_time_utc + timedelta(days=1)  # 24시간 후

query_data = f'''
import "experimental"
from(bucket: "environmentalsensors_data")
  |> range(start: {start_time_utc.strftime('%Y-%m-%dT%H:%M:%SZ')}, stop: {end_time_utc.strftime('%Y-%m-%dT%H:%M:%SZ')})
  |> filter(fn: (r) => r["place"] == "class_a")
  |> filter(fn: (r) => r["measurement"] == "{sensor_name}")
  |> aggregateWindow(every: 2m, fn: mean, createEmpty: false)
  |> map(fn: (r) => ({{r with _time: experimental.addDuration(d: 9h, to: r._time)}}))
  |> keep(columns: ["_time", "_value", "place", "location", "device"])
'''

# CSV 변환
df_sensor_data = query_to_dataframe(client, query_data)
df_sensor_data_fix = update_location(df_sensor_data, location_mapping)
print(df_sensor_data_fix.head())
save_csv(df_sensor_data_fix, input_datetime.strftime("%m_%d")+"_"+sensor_name+"_data.csv", "all_data/"+sensor_name+"/")

# client 종료
client.close()

                  time  value    place             location            device
0  2024-04-15 00:02:00   24.0  class_a  bottom_right_corner  24e124126d152969
1  2024-04-15 00:04:00   24.0  class_a  bottom_right_corner  24e124126d152969
2  2024-04-15 00:06:00   24.0  class_a  bottom_right_corner  24e124126d152969
3  2024-04-15 00:08:00   23.8  class_a  bottom_right_corner  24e124126d152969
4  2024-04-15 00:10:00   23.8  class_a  bottom_right_corner  24e124126d152969


In [36]:
# 클라이언트 생성 및 쿼리 실행
client = create_client(url, token, org)

# 한국 시간대 설정
korea_tz = pytz.timezone('Asia/Seoul')

# 사용자로부터 입력받은 날짜와 시간
input_date_str = "2024-04-30 00:00:00"
input_format = "%Y-%m-%d %H:%M:%S"

# 입력 날짜를 datetime 객체로 변환
input_datetime = datetime.strptime(input_date_str, input_format)

# 한국 시간대로 localize
localized_kst = korea_tz.localize(input_datetime)

# UTC로 변환
start_time_utc = localized_kst.astimezone(pytz.utc)
end_time_utc = start_time_utc + timedelta(days=1)  # 24시간 후

# 전력(W) 조회(class a : main) Flux 쿼리
query_power_device_data = f'''
import "experimental"
from(bucket: "powermetrics_data")
  |> range(start: {start_time_utc.strftime('%Y-%m-%dT%H:%M:%SZ')}, stop: {end_time_utc.strftime('%Y-%m-%dT%H:%M:%SZ')})
  |> filter(fn: (r) => r["phase"] == "total")
  |> filter(fn: (r) => r["description"] == "w")
  |> filter(fn: (r) => r["place"] == "class_a")
  |> filter(fn: (r) => r["location"] != "main" and r["location"] != "outdoor_unit_room_light")
  |> aggregateWindow(every: 2m, fn: mean, createEmpty: false)
  |> map(fn: (r) => ({{r with _time: experimental.addDuration(d: 9h, to: r._time)}}))
  |> keep(columns: ["_time", "_value", "place", "location", "device"])
'''

# 전력 CSV 생성
df_power = query_to_dataframe(client, query_power_device_data)
print(df_power.head(2))
save_csv(df_power, input_datetime.strftime("%m_%d")+"_power_device_data.csv", "all_data/power/device/")

# 전력 사용량 조회(class a : main) Flux 쿼리
query_power_usage_device_data = f'''
import "experimental"
from(bucket: "powermetrics_data")
  |> range(start: {start_time_utc.strftime('%Y-%m-%dT%H:%M:%SZ')}, stop: {end_time_utc.strftime('%Y-%m-%dT%H:%M:%SZ')})
  |> filter(fn: (r) => r["phase"] == "kwh")
  |> filter(fn: (r) => r["place"] == "class_a")
  |> filter(fn: (r) => r["description"] == "sum")
  |> filter(fn: (r) => r["location"] != "main" and r["location"] != "outdoor_unit_room_light")
  |> aggregateWindow(every: 2m, fn: mean, createEmpty: false)
  |> map(fn: (r) => ({{r with _time: experimental.addDuration(d: 9h, to: r._time)}}))
  |> keep(columns: ["_time", "_value", "place", "location", "device"])
'''

# 전력 사용량 CSV 생성
df_power_usage = query_to_dataframe(client, query_power_usage_device_data)
save_csv(df_power_usage, input_datetime.strftime("%m_%d")+"_power_usage_device_data.csv", "all_data/power_usage/device/")
print(df_power_usage.head(2))

# client 종료
client.close()

                  time  value    place        location     device
0  2024-04-30 00:02:00   77.5  class_a  ac_indoor_unit  gems-3500
1  2024-04-30 00:04:00   79.0  class_a  ac_indoor_unit  gems-3500
                  time  value    place        location     device
0  2024-04-30 00:02:00   38.7  class_a  ac_indoor_unit  gems-3500
1  2024-04-30 00:04:00   38.7  class_a  ac_indoor_unit  gems-3500
